이미지 비슷한것끼리 묶어서 데이터 줄이는 용도

In [ ]:
from transformers import CLIPProcessor, CLIPModel

clip_model = CLIPModel.from_pretrained("geolocal/StreetCLIP")
clip_model.to('cuda')
clip_processor = CLIPProcessor.from_pretrained("geolocal/StreetCLIP")

In [ ]:
def get_img_embeddings(paths):
    image_paths = sorted(glob(f'{paths}/*.png'))
    image_features = []
    for i in tqdm(range(0,len(image_paths), BATCH_SIZE)):
        image_paths_batch = image_paths[i:i+BATCH_SIZE]
        images = [Image.open(image_path) for image_path in image_paths_batch]
        pixel_values = clip_processor.image_processor(images=images, return_tensors='pt')['pixel_values'].to('cuda')
        
        with torch.no_grad():
            image_features_row = clip_model.get_image_features(pixel_values).cpu().numpy()
        image_features.append(image_features_row)
        
    embeddings = np.vstack(image_features)  # 모든 배치를 하나의 배열로 통합
    embeddings = normalize(embeddings, norm="l2") # L2 정규화
    return embeddings

In [ ]:
import gc
import torch
gc.collect()
torch.cuda.empty_cache()
BATCH_SIZE = 128